In [1]:
## Import libraries

# Basics
import numpy as np
import pandas as pd
import os
from datetime import datetime
from collections import defaultdict

# Plotting / Mapping
import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap, BoundaryNorm, colors
# from IPython.display import display
# from PIL import Image
# import branca.colormap as cm
import plotly.express as px
# import folium
# from folium.raster_layers import ImageOverlay

# Geo
from dataretrieval import nwis
import rasterio
from rasterio.mask import mask
# from rasterio.plot import show, reshape_as_image
# from rasterio.transform import from_bounds, array_bounds
# from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
from shapely.geometry import mapping, box
# import richdem as rd

# Sklearn
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error




Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
import pickle

# Data Cleaning
1. Create empty np.array for each stream and store them in one dictionary
2. Extract 25 values for each stream from GFDS data
    - Repeat
        - Open one geotiff file
        - Extract 25 values for all the streams
        - Save the 25 values into each stream array

In [3]:
# Read HCDN info
hcdn = pd.read_excel('HCDN-2009_Station_Info.xlsx')
hcdn.head(3)

/Users/asumi/opt/anaconda3/envs/Research/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


,STATION ID,STATION NAME,CLASS,AGGECOREGION,DRAIN_SQKM,HUC02,LAT_GAGE,LONG_GAGE,STATE,ACTIVE09,FLOWYRS_1900_2009,FLOWYRS_1950_2009,FLOWYRS_1990_2009
0,1013500,"Fish River near Fort Kent, Maine",Ref,NorthEast,2252.6960,1,47.237394,-68.582642,ME,yes,85,60,20
1,1022500,"Narraguagus River at Cherryfield, Maine",Ref,NorthEast,573.6006,1,44.607972,-67.935242,ME,yes,61,60,20
2,1030500,"Mattawamkeag River near Mattawamkeag, Maine",Ref,NorthEast,3676.1720,1,45.500975,-68.305956,ME,yes,75,60,20


In [19]:
# Define GFDS path 
dir_path = '/Users/asumi/Downloads/AvgMagTiffs/2018'
file_list = os.listdir(dir_path)
file_list = list(set(file_list))
file_list.remove('.DS_Store')
file_list.sort()
len(file_list)

365

In [51]:
# Extract GFDS for all the streams

# Create empty dictionary to store data
# Dict key is set to gage(station) no.
stream_patches = defaultdict(list)

for item in file_list:
    
    # file path for RS data
    file_path = os.path.join(dir_path, item)
    # save file date as string (this process is specifict to my data)
    date_str = item[33:41]

    with rasterio.open(file_path) as src:

        for idx, row in hcdn.iterrows():
            lat, lon = row['LAT_GAGE'], row['LONG_GAGE']
            station_id = row['STATION ID']

            # identify row and column of gage location
            row, col = src.index(lon, lat)
            # create window of 5 x 5 cells centered to gage location
            window = rasterio.windows.Window(col-2, row-2, 5, 5)

            # filter the raster file with the window
            patch = src.read(1, window=window)

            # cutoff outliers (this process is specifict to my data)
            patch = patch.astype(np.float32)
            patch[patch < -30000] = np.nan

            # flattern 5x5 cells into a list
            patch_list = patch.flatten().tolist()
            patch_list.insert(0,date_str)

            # store the cell values into dictionary
            stream_patches[station_id].append(patch_list)

# Convert list to np.array
for station_id in stream_patches:
    stream_patches[station_id] = np.array(stream_patches[station_id])

    

In [ ]:
# Store the dict to pickle
with open("stream_patch_2018.pkl", 'wb') as f:
    pickle.dump(stream_patches, f)

In [4]:
# Read pickle
with open('stream_patch_2018.pkl', 'rb') as f:
    stream_patches = pickle.load(f)

In [9]:
stream_patches[1030500]

array([['20180101', '2742.0', '2894.0', ..., '-912.0', '-941.0',
        '-1380.0'],
       ['20180102', '2677.0', '2585.0', ..., '-894.0', '-1055.0',
        '-1432.0'],
       ['20180103', '3589.0', '3623.0', ..., '-576.0', '-834.0',
        '-1343.0'],
       ...,
       ['20181229', '1353.0', '1188.0', ..., '842.0', '1111.0', '32.0'],
       ['20181230', '2289.0', '1299.0', ..., '738.0', '660.0', '-54.0'],
       ['20181231', '2354.0', '1521.0', ..., '1055.0', '1001.0', '92.0']],
      dtype='<U32')